# dbachecks PR check on Beard NUC - MultipleInstances

Rob uses this process to make sure that changes will work in an AD estate with a variety of versions

First we must check the status of the local source control and ensure that we are in the PR branch

In [1]:
cd D:\OneDrive\Documents\GitHub\dbachecks
    git status
    

On branch new-json
Your branch is ahead of 'origin/new-json' by 162 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   internal/configurations/DbcCheckDescriptions.json

no changes added to commit (use "git add" and/or "git commit -a")


First we need to import the module from source control - I normally use verbose to see how it is going


In [2]:
Import-Module .\dbachecks.psd1
$null= Reset-DbcConfig 

# Make sure that the check and config have been written to the json file

In [3]:
Get-DbcCheck QueryStoreEnabled  | fl
Get-DbcCheck QueryStoreDisabled | fl


Group       : Database
Type        : Sqlinstance
UniqueTag   : QueryStoreEnabled
AllTags     : QueryStoreEnabled, Medium, Database
Config      : app.sqlinstance database.querystoreenabled.excludedb 
Description : Tests to see Query Store is enabled on each database except those specified.



Group       : Database
Type        : Sqlinstance
UniqueTag   : QueryStoreDisabled
AllTags     : QueryStoreDisabled, Medium, Database
Config      : app.sqlinstance database.querystoredisabled.excludedb 
Description : Tests to see Query Store is disabled on each database except those specified.




In [5]:
Get-DbcConfig database.querystoreenabled.excludedb | fl
Get-DbcConfig database.querystoredisabled.excludedb | fl


Name        : database.querystoreenabled.excludedb
Value       : {}
Description : A List of databases that we do not want to check for Query Store enabled



Name        : database.querystoredisabled.excludedb
Value       : {}
Description : A List of databases that we do not want to check for Query Store disabled




In [7]:
$Sqlinstances = (Get-VM -ComputerName BEARDNUC, BEARDNUC2 -Name *SQL* | Where State -eq 'Running').Name

Invoke-DbcCheck -SqlInstance $Sqlinstances -Check QueryStoreEnabled, QueryStoredisabled

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1' with Tags QueryStoreEnabled', 'QueryStoredisabled

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1

  Describing Query Store Enabled

    Context Testing to see if Query Store is enabled on SQL2005Ser2003
      [!] Database AdventureWorks should have Query Store enabled on SQL2005Ser2003, is skipped 0ms
      [!] Database AdventureWorksDW should have Query Store enabled on SQL2005Ser2003, is skipped 0ms
      [!] Database Aerosmith should have Query Store enabled on SQL2005Ser2003, is skipped 0ms
      [!] Database BlackSabbath should have Query Store enabled on SQL2005Ser2003, is skipped 0ms
      [!] Database DBA-Admin should have Query Store enabled on SQL2005Ser2003, is skipped 0ms
      [!] Databas

Set the config and run again

In [10]:
$Sqlinstances = 'SQL2019N5','SQL2017N5'
Set-DbcConfig -Name database.querystoredisabled.excludedb -Value 'Model','SqlWatch'
Set-DbcConfig -Name database.querystoreenabled.excludedb -Value 'Model','SqlWatch'
Invoke-DbcCheck -SqlInstance $Sqlinstances -Check QueryStoreEnabled, QueryStoredisabled


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1' with Tags QueryStoreEnabled', 'QueryStoredisabled

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1

  Describing Query Store Enabled

    Context Testing to see if Query Store is enabled on SQL2019N5

  Describing Query Store Disabled

    Context Testing to see if Query Store is disabled on SQL2019N5

  Describing Query Store Enabled

    Context Testing to see if Query Store is enabled on SQL2017N5
      [-] Database ADonutFactory should have Query Store enabled on SQL2017N5 66ms
        Expected 1, because We expect the Query Store to be enabled in ADonutFactory on [SQL2017N5], but got 0.
        113:     @(Get-DbaDbQueryStoreOption -SqlInstance $Instance -Database $Database | Where-Object {$_.Actua

Lets enable it on a database and check again

In [12]:
Set-DbaDbQueryStoreOption -SqlInstance SQL2017N5 -Database ADonutFactory -State ReadWrite


ComputerName                          : SQL2017N5
InstanceName                          : MSSQLSERVER
SqlInstance                           : SQL2017N5
Database                              : ADonutFactory
ActualState                           : ReadWrite
DataFlushIntervalInSeconds            : 900
StatisticsCollectionIntervalInMinutes : 60
MaxStorageSizeInMB                    : 100
CurrentStorageSizeInMB                : 0
QueryCaptureMode                      : All
SizeBasedCleanupMode                  : Auto
StaleQueryThresholdInDays             : 30
MaxPlansPerQuery                      : 200
WaitStatsCaptureMode                  : ON




In [18]:
$Sqlinstances = 'SQL2017N5'
Set-DbcConfig -Name database.querystoredisabled.excludedb -Value 'Model','SqlWatch', 'ASausageFactory','ASockFactory', 'BeardNotebooks', 'SSISDB'
Set-DbcConfig -Name database.querystoreenabled.excludedb -Value 'Model','SqlWatch', 'ASausageFactory','ASockFactory', 'BeardNotebooks', 'SSISDB'
Invoke-DbcCheck -SqlInstance $Sqlinstances -Check QueryStoreEnabled, QueryStoredisabled


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1' with Tags QueryStoreEnabled', 'QueryStoredisabled

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1

  Describing Query Store Enabled

    Context Testing to see if Query Store is enabled on SQL2017N5
      [+] Database ADonutFactory should have Query Store enabled on SQL2017N5 60ms

  Describing Query Store Disabled

    Context Testing to see if Query Store is disabled on SQL2017N5
      [-] Database ADonutFactory should have Query Store disabled on SQL2017N5 132ms
        Expected 'OFF', because We expect the Query Store to be disabled in ADonutFactory on [SQL2017N5], but got ReadWrite.
        120:     (Get-DbaDbQueryStoreOption -SqlInstance $Instance -Database $Database).ActualState | Should -Be 'O